## IMDb

At Fast.ai we have introduced a new module called fastai.text which replaces the torchtext library that was used in our 2018 dl1 course. The fastai.text module also supersedes the fastai.nlp library but retains many of the key functions.

In [1]:
from fastai.text import *
from fastai.core import num_cpus, partition_by_cores
import html
from pathlib import Path
import numpy as np
import csv
import pandas as pd
from collections import Counter, defaultdict
from itertools import chain
from nltk.corpus import brown
import os

from gensim.corpora import Dictionary
from gensim.models import Word2Vec
from typing import Callable, List, Collection
from concurrent.futures.process import ProcessPoolExecutor

The Fastai.text module introduces several custom tokens.

We need to download the IMDB large movie reviews from this site: http://ai.stanford.edu/~amaas/data/sentiment/
Direct link : [Link](http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz) and untar it into the PATH location. We use pathlib which makes directory traveral a breeze.

In [6]:
PATH=Path('/media/discoD/Corpora/Portuguese Wikipedia Dump/text/AA/')

## Standardize format

The imdb dataset has 3 classes. positive, negative and unsupervised(sentiment is unknown). 
There are 75k training reviews(12.5k pos, 12.5k neg, 50k unsup)
There are 25k validation reviews(12.5k pos, 12.5k neg & no unsup)

Refer to the README file in the imdb corpus for further information about the dataset.

In [3]:
class VocabularyTokenizer():
    "Put together rules, a tokenizer function and a language to tokenize text with multiprocessing."
    def __init__(self, tok_func:Callable=SpacyTokenizer, lang:str='pt', n_cpus:int=None):
        self.tok_func,self.lang = tok_func,lang
        self.n_cpus = n_cpus or num_cpus()//2

    def process_text(self, t:str, tok:BaseTokenizer) -> List[str]:
        "Processe one text `t` with tokenizer `tok`."
        return tok.tokenizer(t)

    def _process_all_1(self, texts:Collection[str]) -> List[List[str]]:
        "Process a list of `texts` in one process."
        tok = self.tok_func(self.lang)
        return [self.process_text(t, tok) for t in texts]

    def process_all(self, texts:Collection[str]) -> List[List[str]]:
        "Process a list of `texts`."
        if self.n_cpus <= 1: return self._process_all_1(texts)
        with ProcessPoolExecutor(self.n_cpus) as e:
            return sum(e.map(self._process_all_1, partition_by_cores(texts, self.n_cpus)), [])

In [7]:
def save_texts(paths, filename, lang):
    CLASSES = ['unsup']
    file_count = 0
    filename = filename + '_' + lang + '.csv'
    if os.path.isfile(filename):
        os.remove(filename)
    with open(filename, 'a') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONE, escapechar='\\')
        for idx,label in enumerate(CLASSES):
            for path in paths:
                for fname in (path).glob('*'):
                    file_count += 1
                    print('writing from %s' % fname)
                    [writer.writerow([line, idx]) for line in fname.open('r', encoding='utf-8').read().split('\n')]
    print('%d texts saved to %s' % (file_count, filename))

In [8]:
save_texts([PATH], 'wiki_00', 'pt')

writing from /media/discoD/Corpora/Portuguese Wikipedia Dump/text/AA/wiki_00
1 texts saved to wiki_00_pt.csv


In [4]:
def get_tokens(filename):
    data = pd.read_csv(filename, header=None, escapechar='\\', chunksize=500000)
    for idx, df in enumerate(data):
        print(idx)
        yield VocabularyTokenizer().process_all(df[0].astype(str))

In [9]:
freq_full = Counter(p for o in chain.from_iterable(get_tokens('wiki_00_pt.csv')) for p in o)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [10]:
sum(freq_full.values())

279449748

In [15]:
palavras = [palavra for palavra, contagem in freq_full.most_common()]
palavras

[',',
 'de',
 '.',
 'a',
 'e',
 '"',
 'o',
 'do',
 'em',
 'da',
 'que',
 'um',
 'uma',
 ')',
 '(',
 'com',
 'é',
 'no',
 'para',
 'na',
 'foi',
 'os',
 'por',
 'como',
 'O',
 'A',
 'dos',
 'se',
 'as',
 'sua',
 'mais',
 ':',
 'à',
 'Em',
 'seu',
 'não',
 'das',
 'ou',
 'pela',
 'também',
 'pelo',
 'ser',
 'd',
 'entre',
 'são',
 'anos',
 'ele',
 'era',
 'mas',
 '-',
 ';',
 'nos',
 'foram',
 'No',
 'cidade',
 'até',
 'seus',
 'onde',
 'Os',
 '/',
 "'",
 'sobre',
 'tem',
 'km',
 'ano',
 'São',
 'quando',
 'parte',
 'sendo',
 'nas',
 'primeiro',
 'mesmo',
 'dois',
 'primeira',
 'suas',
 'Brasil',
 'durante',
 'depois',
 's',
 'Foi',
 'nome',
 'outros',
 'ela',
 'As',
 'ainda',
 'dia',
 'grande',
 'região',
 'este',
 'álbum',
 'família',
 'está',
 '%',
 'já',
 'muito',
 'The',
 'pode',
 'maior',
 'após',
 '--',
 '²',
 'apenas',
 'vez',
 'esta',
 'É',
 '1',
 'Ele',
 'área',
 'três',
 'estado',
 'Rio',
 'Na',
 'ter',
 'forma',
 'filme',
 'n',
 'então',
 'contra',
 'duas',
 'sido',
 'segundo'

In [16]:
len(palavras)

2386970

In [19]:
palavras.insert(0, '<UNK>')
palavras.insert(0, '<S>')
palavras.insert(0, '</S>')
palavras

['</S>',
 '<S>',
 '<UNK>',
 ',',
 'de',
 '.',
 'a',
 'e',
 '"',
 'o',
 'do',
 'em',
 'da',
 'que',
 'um',
 'uma',
 ')',
 '(',
 'com',
 'é',
 'no',
 'para',
 'na',
 'foi',
 'os',
 'por',
 'como',
 'O',
 'A',
 'dos',
 'se',
 'as',
 'sua',
 'mais',
 ':',
 'à',
 'Em',
 'seu',
 'não',
 'das',
 'ou',
 'pela',
 'também',
 'pelo',
 'ser',
 'd',
 'entre',
 'são',
 'anos',
 'ele',
 'era',
 'mas',
 '-',
 ';',
 'nos',
 'foram',
 'No',
 'cidade',
 'até',
 'seus',
 'onde',
 'Os',
 '/',
 "'",
 'sobre',
 'tem',
 'km',
 'ano',
 'São',
 'quando',
 'parte',
 'sendo',
 'nas',
 'primeiro',
 'mesmo',
 'dois',
 'primeira',
 'suas',
 'Brasil',
 'durante',
 'depois',
 's',
 'Foi',
 'nome',
 'outros',
 'ela',
 'As',
 'ainda',
 'dia',
 'grande',
 'região',
 'este',
 'álbum',
 'família',
 'está',
 '%',
 'já',
 'muito',
 'The',
 'pode',
 'maior',
 'após',
 '--',
 '²',
 'apenas',
 'vez',
 'esta',
 'É',
 '1',
 'Ele',
 'área',
 'três',
 'estado',
 'Rio',
 'Na',
 'ter',
 'forma',
 'filme',
 'n',
 'então',
 'contra',
 

In [18]:
sum(freq_full.values())

267312748

In [22]:
with open('vocabulario.txt', 'w') as vocab:
    for palavra in palavras:
        vocab.write(palavra + '\n')
vocab.close()

In [22]:
singletons = [palavra for palavra, contagem in freq_full.most_common() if contagem == 1]
print(len(singletons))
singletons

1177677


['intermediável',
 'Septicemias',
 'pneumocitose',
 'Fiameta',
 'Stors',
 'Bellovaci',
 'Silvanecti',
 'Caméliacencis',
 '1,7920987',
 '0,1602156',
 '138,63',
 '18.124',
 'Mãe-de-santo',
 'calculando-a',
 'Zeischrift',
 'Europaeisches',
 '0.08880340',
 '4.00539º.',
 'Kamukirimusi',
 'figureskater',
 'paratiarca',
 'Nacianceno',
 'Pandemo',
 'Briles',
 'Evanof',
 'Galberto',
 'Tokiti',
 'choco-anão',
 'sepiola',
 'casseron',
 'chipiron',
 'Teiro',
 'Catersport',
 '2.6.21',
 'Unthing',
 'levantar-se-ão',
 'penetrarão',
 '7.237',
 'Žofie',
 'Josefína',
 'Albína',
 'hraběnka',
 'Chotková',
 'Chotkova',
 'Vojnína',
 'iθ',
 'Torpedo-Viktorya',
 'Gilles-Gaston',
 'circuncisá-lo',
 'esfigmomanometria',
 'Gavião-do-banhado',
 'Inocent',
 'Leibhusaren-Regiment',
 'Baumbauer',
 'hones',
 '24.02.2012',
 'Rashaayda',
 'الرشايدة',
 'Rachaídas',
 'wa-l-',
 'ramadanidas',
 'niyābāt',
 'al-Bira',
 'al-Ruha',
 '三花控股集团有限公司',
 'Markethill',
 '198.973',
 'Rahavard',
 '6.091.870',
 'Turbevill',
 '1c0',
 '35